# 04_Data_Merge 

## Overview – Data Integration and Merge Validation
This notebook integrates hospital-level staffing, quality ratings, and regional nursing workforce
datasets into a single analysis-ready dataset.

The primary objective is to construct a unified hospital-level file that links CMS Hospital
Provider Cost Report staffing measures, CMS Hospital Quality Ratings, and Bureau of Labor
Statistics (BLS) nursing workforce indicators. Merge quality, coverage, and data loss are
explicitly evaluated to ensure transparency and reproducibility.

## Data Sources Integrated
This notebook combines the following cleaned datasets produced in earlier steps:

- **CMS Hospital Provider Cost Report**  
  Hospital-level staffing capacity and utilization measures (e.g., FTEs, beds, patient days)

- **CMS Hospital Quality Ratings**  
  Overall hospital star ratings and performance group summaries

- **BLS Nursing Workforce Data (MSA & State)**  
  Regional registered nurse employment levels, nurses per 1,000 jobs, and wage measures


## 1. Load Cleaned Input Datasets
This section loads the cleaned outputs from prior EDA notebooks and verifies dataset dimensions
to confirm expected coverage prior to merging.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 250)
pd.set_option("display.width", 160)

# --- Project paths (01-style, robust) ---
ROOT = Path.cwd().resolve()

# If you're already inside /work/src/notebooks, climb to /work
if ROOT.name == "notebooks" and ROOT.parent.name == "src":
    ROOT = ROOT.parents[1]          # /work
elif ROOT.name == "src":
    ROOT = ROOT.parent              # /work

SRC = ROOT / "src"
RAW_DIR = SRC / "data" / "raw"
CLEAN_DIR = SRC / "data" / "clean"
INTERIM_DIR = SRC / "data" / "interim"

RAW_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

COST_CLEAN_PATH = CLEAN_DIR / "cost_report_clean.csv"
RATINGS_CLEAN_PATH = CLEAN_DIR / "ratings_clean.csv"
BLS_MSA_CLEAN_PATH = CLEAN_DIR / "bls_msa_rn_clean.csv"
BLS_STATE_CLEAN_PATH = CLEAN_DIR / "bls_state_rn_clean.csv"

ANALYSIS_OUT_PATH = INTERIM_DIR / "analysis_dataset.csv"

print("CWD:", Path.cwd())
print("Resolved ROOT:", ROOT)
print("Reading:", COST_CLEAN_PATH)
print("Reading:", RATINGS_CLEAN_PATH)
print("Reading:", BLS_MSA_CLEAN_PATH)
print("Reading:", BLS_STATE_CLEAN_PATH)
print("Writing:", ANALYSIS_OUT_PATH)

for p in [COST_CLEAN_PATH, RATINGS_CLEAN_PATH, BLS_MSA_CLEAN_PATH, BLS_STATE_CLEAN_PATH]:
    assert p.exists(), f"Missing file: {p}"

CWD: /voc/work/src/notebooks
Resolved ROOT: /voc/work
Reading: /voc/work/src/data/clean/cost_report_clean.csv
Reading: /voc/work/src/data/clean/ratings_clean.csv
Reading: /voc/work/src/data/clean/bls_msa_rn_clean.csv
Reading: /voc/work/src/data/clean/bls_state_rn_clean.csv
Writing: /voc/work/src/data/interim/analysis_dataset.csv


In [3]:
cost = pd.read_csv(COST_CLEAN_PATH, low_memory=False)
ratings = pd.read_csv(RATINGS_CLEAN_PATH, low_memory=False)
bls_msa = pd.read_csv(BLS_MSA_CLEAN_PATH, low_memory=False)
bls_state = pd.read_csv(BLS_STATE_CLEAN_PATH, low_memory=False)

print("cost:", cost.shape)
print("ratings:", ratings.shape)
print("bls_msa:", bls_msa.shape)
print("bls_state:", bls_state.shape)

cost: (6103, 12)
ratings: (5421, 15)
bls_msa: (391, 11)
bls_state: (54, 11)


## 2. Standardize Merge Keys
Hospital and geographic identifiers are standardized to ensure consistent merging across datasets.

Key standardizations include:
- Zero-padding CMS Certification Numbers (CCNs)
- Converting Medicare CBSA codes to string format for MSA-level joins
- Harmonizing state abbreviations across CMS and BLS datasets

In [4]:
# Hospital IDs
cost["Provider CCN"] = cost["Provider CCN"].astype(str).str.strip().str.zfill(6)
ratings["Facility ID"] = ratings["Facility ID"].astype(str).str.strip().str.zfill(6)

# State keys
if "State Code" in cost.columns:
    cost["State Code"] = cost["State Code"].astype(str).str.strip()

ratings["State"] = ratings["State"].astype(str).str.strip()
bls_state["PRIM_STATE"] = bls_state["PRIM_STATE"].astype(str).str.strip()

# BLS MSA AREA is the join key, keep it as string
bls_msa["AREA"] = bls_msa["AREA"].astype(str).str.strip()

# --- CBSA key on cost ---
# Prefer cost["cbsa"] if it exists (you now create it in 01)
if "cbsa" in cost.columns:
    cost["cbsa_str"] = (
        cost["cbsa"]
        .astype("string")
        .str.strip()
        .replace({"nan": pd.NA, "": pd.NA})
    )
else:
    # fallback: derive from Medicare CBSA Number
    cost["Medicare CBSA Number"] = pd.to_numeric(cost["Medicare CBSA Number"], errors="coerce")
    cost["cbsa_str"] = cost["Medicare CBSA Number"].astype("Int64").astype("string")
    cost.loc[cost["cbsa_str"] == "<NA>", "cbsa_str"] = pd.NA

print("CBSA key missing %:", cost["cbsa_str"].isna().mean())
print("Example CBSA keys:", cost["cbsa_str"].dropna().head(10).tolist())


CBSA key missing %: 0.013436015074553498
Example CBSA keys: ['99911.0', '29404.0', '22220.0', '18580.0', '99942.0', '41540.0', '99903.0', '12100.0', '30980.0', '44180.0']


## 3. Merge Cost Report and Quality Ratings Data
CMS Hospital Provider Cost Report data are merged with CMS Hospital Quality Ratings using the
CMS Certification Number as the hospital-level join key.

Merge coverage and uniqueness are evaluated to quantify how many hospitals are retained for
analysis after linking staffing and quality data.

In [5]:
m1 = cost.merge(
    ratings,
    left_on="Provider CCN",
    right_on="Facility ID",
    how="inner"
)

print("Cost rows:", len(cost), "Unique CCNs:", cost["Provider CCN"].nunique())
print("Ratings rows:", len(ratings), "Unique Facility IDs:", ratings["Facility ID"].nunique())
print("Merged cost+ratings rows:", len(m1), "Unique CCNs merged:", m1["Provider CCN"].nunique())

Cost rows: 6103 Unique CCNs: 6040
Ratings rows: 5421 Unique Facility IDs: 5421
Merged cost+ratings rows: 5204 Unique CCNs merged: 5162


## 4. Merge MSA-Level BLS Nursing Workforce Data
MSA-level BLS nursing workforce measures are merged using Medicare CBSA codes where available.

This step provides fine-grained regional labor market context but is expected to have incomplete
coverage due to BLS suppression rules, particularly in smaller or rural MSAs.

In [6]:
msa_cols = [c for c in ["AREA", "AREA_TITLE", "PRIM_STATE", "TOT_EMP", "JOBS_1000", "A_MEAN", "H_MEAN"] if c in bls_msa.columns]
bls_msa_small = bls_msa[msa_cols].copy()

# rename to *_msa
rename_msa = {c: f"{c}_msa" for c in bls_msa_small.columns if c != "AREA"}
bls_msa_small = bls_msa_small.rename(columns=rename_msa)

m2 = m1.merge(
    bls_msa_small,
    left_on="cbsa_str",
    right_on="AREA",
    how="left"
)

print("After MSA merge rows:", len(m2))
if "TOT_EMP_msa" in m2.columns:
    print("MSA match rate (TOT_EMP_msa not null):", m2["TOT_EMP_msa"].notna().mean())

# quick diagnostics if match rate is unexpectedly low
print("Unique CBSAs in hospitals:", m1["cbsa_str"].nunique())
print("Unique AREA in BLS MSA:", bls_msa_small["AREA"].nunique())
print("Sample hospital CBSA keys:", m1["cbsa_str"].dropna().head(10).tolist())
print("Sample BLS AREA keys:", bls_msa_small["AREA"].dropna().head(10).tolist())

After MSA merge rows: 5204
MSA match rate (TOT_EMP_msa not null): 0.0
Unique CBSAs in hospitals: 467
Unique AREA in BLS MSA: 391
Sample hospital CBSA keys: ['29404.0', '18580.0', '38060.0', '99923.0', '12260.0', '35660.0', '99923.0', '99934.0', '99934.0', '28940.0']
Sample BLS AREA keys: ['10180', '10380', '10420', '10500', '10540', '10580', '10740', '10780', '10900', '11020']


## 5. Merge State-Level BLS Nursing Workforce Data (Fallback)
To address gaps in MSA-level coverage, state-level BLS nursing workforce measures are merged as a
fallback using state abbreviations.

This ensures near-complete coverage of hospitals with at least state-level workforce context,
while preserving MSA-level measures where available.

In [7]:
state_cols = [c for c in ["PRIM_STATE", "TOT_EMP", "JOBS_1000", "A_MEAN", "H_MEAN"] if c in bls_state.columns]
bls_state_small = bls_state[state_cols].copy()

rename_state = {c: f"{c}_state" for c in bls_state_small.columns if c != "PRIM_STATE"}
bls_state_small = bls_state_small.rename(columns=rename_state)

m3 = m2.merge(
    bls_state_small,
    left_on="State",       # from ratings
    right_on="PRIM_STATE",
    how="left"
)

print("After State merge rows:", len(m3))
print("State match rate (TOT_EMP_state not null):", m3["TOT_EMP_state"].notna().mean())

After State merge rows: 5204
State match rate (TOT_EMP_state not null): 0.9998078401229823


## 6. Construct Best-Available Workforce Measures
For each nursing workforce indicator, a “best-available” version is created that prioritizes
MSA-level measures when present and defaults to state-level values otherwise.

This approach balances geographic precision with completeness and minimizes data loss due to
suppression.

In [8]:
m3["rn_emp_best"] = m3["TOT_EMP_msa"].fillna(m3["TOT_EMP_state"]) if "TOT_EMP_msa" in m3.columns else m3["TOT_EMP_state"]
m3["rn_jobs_per_1000_best"] = m3["JOBS_1000_msa"].fillna(m3["JOBS_1000_state"]) if "JOBS_1000_msa" in m3.columns else m3["JOBS_1000_state"]
m3["rn_wage_best"] = m3["A_MEAN_msa"].fillna(m3["A_MEAN_state"]) if "A_MEAN_msa" in m3.columns else m3["A_MEAN_state"]
m3["rn_hourly_best"] = m3["H_MEAN_msa"].fillna(m3["H_MEAN_state"]) if "H_MEAN_msa" in m3.columns else m3["H_MEAN_state"]

print("MSA TOT_EMP available %:", m3["TOT_EMP_msa"].notna().mean() if "TOT_EMP_msa" in m3.columns else "n/a")
print("State TOT_EMP available %:", m3["TOT_EMP_state"].notna().mean())
print("Best-available TOT_EMP %:", m3["rn_emp_best"].notna().mean())

MSA TOT_EMP available %: 0.0
State TOT_EMP available %: 0.9998078401229823
Best-available TOT_EMP %: 0.9998078401229823


## 7. Save Final Analysis Dataset
The final merged dataset is saved in a single analysis-ready file containing staffing measures,
quality outcomes, and regional workforce context.

This dataset serves as the input for statistical analysis and visualization in the final project
notebook.

In [9]:
m3.to_csv(ANALYSIS_OUT_PATH, index=False)
print("Saved:", ANALYSIS_OUT_PATH, m3.shape)

Saved: /voc/work/src/data/interim/analysis_dataset.csv (5204, 44)
